#Ejemplo: Método de Newton-Raphson para la Solución de Flujos de Potencia



unif-ejemplo1.svg

##Tabla de datos de nodos:

| Nodos | Tipo de nodo | Tensión nominal (kV) | Potencia activa inyectada (MW) | Potencia reactiva inyectada (MVAr) | Magnitud de voltaje (p.u.) | Ángulo de voltaje (p.u.) |
|-------|--------------|----------------------|--------------------------------|------------------------------------|----------------------------|--------------------------|
| 1     | Referencia   | 100                  | --                             | --                                 | 1.02                       | 0.0                      |
| 2     | Generación   | 100                  | 50                             | --                                 | 1.02                       | 0.0                      |
| 3     | Carga        | 100                  | -100                           | -60                                | 1.02                       | 0.0                      |

## Solución
*Inicialmente, se definen los datos del sistema en valores por unidad referenciados a una base común de 100 MVA y 100 kV.*

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def rad(deg):
  # Convertir grados en radianes
  return deg*np.pi/180

def asvoltage(Mag,Ang):
  # Convertir vectores de magnitud y ángulo en vector de fasores
  # Mag: vector de magnitudes de voltaje
  # Ang: vector de ángulos de voltaje
  v=np.zeros((len(Mag),1),dtype=complex)
  for i in range(len(Mag)):
    v[i]=Mag[i]*(np.cos(Ang[i])+1j*np.sin(Ang[i]))
  return v

# Definir matriz de admitancias
n=3
Ybus=np.array([(15-35j,-10+20j,-5+15j),(-10+20j,30-60j,-20+40j),(-5+15j,-20+40j,25-55j)],dtype=complex)
B=np.imag(Ybus)
G=np.real(Ybus)
print('Matriz de admitancias:\n',Ybus)

# Valores definidos de potencia
Pdef=np.array([[0.5,-1]])
Qdef=np.array([[-0.6]])

# Valores iniciales de voltaje
vbusMag=np.array([1.02,1.02,1.02])
vbusAng=np.array([rad(0),rad(0),rad(0)])
vbus=asvoltage(vbusMag,vbusAng)
print('Voltajes:\n',vbus)

Matriz de admitancias:
 [[ 15.-35.j -10.+20.j  -5.+15.j]
 [-10.+20.j  30.-60.j -20.+40.j]
 [ -5.+15.j -20.+40.j  25.-55.j]]
Voltajes:
 [[1.02+0.j]
 [1.02+0.j]
 [1.02+0.j]]


Se calculan las potencias inyectadas y los residuos con respecto a las potencias definidas.

In [2]:
# Calcular potencias inyectadas a cada bus
S=np.zeros((n,1),dtype=complex)
for i in range(0,n):
    S[i]=vbus[i]*np.conjugate(np.matmul(Ybus[i,:],vbus))
print('Potencia compleja:\n',S)

# Calcular residuos
def residuos(S,Pdef,Qdef):
  Qcal=np.imag(S)
  Pcal=np.real(S)
  dP=np.swapaxes(Pdef,0,1)-np.real(S)[1:3]
  dQ=Qdef-np.imag(S)[2]
  dPQ=np.concatenate((dP,dQ),axis=0)
  return dPQ
dPQ=residuos(S,Pdef,Qdef)
print('Residuos:',dPQ)

Potencia compleja:
 [[1.81188398e-15+3.62376795e-15j]
 [3.62376795e-15+7.24753590e-15j]
 [0.00000000e+00+7.24753590e-15j]]
Residuos: [[ 0.5]
 [-1. ]
 [-0.6]]


Como los residuos son mayores a la tolerancia (supongamos que es $10^{-6}$), se debe continuar a calcular el Jacobiano y actualizar las variables.

In [3]:
# Se definen los nodos con P conocida y Q conocida
pconocida=[2,3]
qconocida=[3]
Qcal=np.imag(S)
Pcal=np.real(S)

# Ciclo de Newton-Raphson
count=0
print('Iter.   Residuo máximo       Voltajes (p.u.)          Ángulos (rad)')
while ((max(abs(dPQ))>10e-6) & (count<10)):
  # Calcular submatrices del Jacobiano
  H=np.zeros((len(pconocida),len(pconocida)))
  for i in range(len(pconocida)):
    for j in range(len(pconocida)):
        k=pconocida[i]-1
        m=pconocida[j]-1
        if k==m:
          H[i][j]=-Qcal[k]-B[k][k]*(vbusMag[k])**2
        else:
          H[i][j]=vbusMag[k]*vbusMag[m]*(G[k][m]*np.sin(vbusAng[k]-vbusAng[m])-B[k][m]*np.cos(vbusAng[k]-vbusAng[m]))

  N=np.zeros((len(pconocida),len(qconocida)))
  for i in range(len(pconocida)):
    for j in range(len(qconocida)):
      k=pconocida[i]-1
      m=qconocida[j]-1
      if k==m:
          N[i][j]=Pcal[k]+G[k][k]*(vbusMag[k])**2
      else:
          N[i][j]=vbusMag[k]*vbusMag[m]*(G[k][m]*np.cos(vbusAng[k]-vbusAng[m])+B[k][m]*np.sin(vbusAng[k]-vbusAng[m]))

  M=np.zeros((len(qconocida),len(pconocida)))
  for i in range(len(qconocida)):
    for j in range(len(pconocida)):
      k=qconocida[i]-1
      m=pconocida[j]-1
      if k==m:
        M[i][j]=Pcal[k]-G[k][k]*(vbusMag[k])**2
      else:
        M[i][j]=-vbusMag[k]*vbusMag[m]*(G[k][m]*np.cos(vbusAng[k]-vbusAng[m])+B[k][m]*np.sin(vbusAng[k]-vbusAng[m]))

  L=np.zeros((len(qconocida),len(qconocida)))
  for i in range(len(qconocida)):
    for j in range(len(qconocida)):
      k=qconocida[i]-1
      m=qconocida[j]-1
      if k==m:
        L[i][j]=Qcal[k]-B[k][k]*(vbusMag[k])**2
      else:
        L[i][j]=vbusMag[k]*vbusMag[m]*(G[k][m]*np.sin(vbusAng[k]-vbusAng[m])-B[k][m]*np.cos(vbusAng[k]-vbusAng[m]))

  J1=np.concatenate((H,N),axis=1)
  J2=np.concatenate((M,L),axis=1)
  J=np.concatenate((J1,J2),axis=0)

  # Calcular variaciones
  dDV=np.linalg.solve(J, dPQ)

  # Actualizar voltajes
  for i in range(n-1):
    vbusAng[pconocida[i]-1]+=dDV[i]
  for i in range(len(qconocida)):
    vbusMag[qconocida[i]-1]=vbusMag[qconocida[i]-1]*(1+dDV[i+n-1])
  vbus=asvoltage(vbusMag,vbusAng)

  #Actualizar potencias
  for i in range(0,n):
    S[i]=vbus[i]*np.conjugate(np.matmul(Ybus[i,:],vbus))
  Qcal=np.imag(S)
  Pcal=np.real(S)

  #Calular residuos
  dPQ=residuos(S,Pdef,Qdef)

  count+=1
  print(count,'   ',max(dPQ),'    ',vbusMag,' ',vbusAng)



Iter.   Residuo máximo       Voltajes (p.u.)          Ángulos (rad)
1     [0.00424209]      [1.02       1.02       1.00464341]   [ 0.         -0.00795303 -0.01641642]
2     [6.64282688e-07]      [1.02       1.02       1.00434317]   [ 0.         -0.00822119 -0.01677637]


In [4]:
# Potencia inyectada final
print('Potencia activa inyectada \n',Pcal)
print('Potencia reactiva inyectda \n',Qcal)

Potencia activa inyectada 
 [[ 0.50976795]
 [ 0.49999934]
 [-0.99999455]]
Potencia reactiva inyectda 
 [[ 0.0709557 ]
 [ 0.55125208]
 [-0.59999503]]
